<a href="https://colab.research.google.com/github/Ida1Ersi/wine/blob/main/AiDriving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Jan 31 02:08:16 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip -q install ultralytics opencv-python tqdm pyyaml

In [3]:
!pip -q install kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle API configured successfully")

✅ Kaggle API configured successfully


In [9]:
!pip -q install kagglehub


In [10]:
import kagglehub

path = kagglehub.dataset_download("zeuss2k3/mapillary-traffic-sign-dataset")
print("✅ Dataset downloaded to:", path)


100%|██████████| 32.8G/32.8G [26:01<00:00, 22.6MB/s]

Extracting files...


✅ Dataset downloaded to: /root/.cache/kagglehub/datasets/zeuss2k3/mapillary-traffic-sign-dataset/versions/8


In [11]:
!ls -lh "$path"

total 99M
drwxr-xr-x 3 root root 4.0K Jan 31 03:13 'annotations for mtsd in coco format'
drwxr-xr-x 3 root root 4.0K Jan 31 03:13  mtsd_fully_annotated_train_images
drwxr-xr-x 3 root root 4.0K Jan 31 03:19  mtsd_partially_annotated_annotation
drwxr-xr-x 3 root root 4.0K Jan 31 03:19  mtsd_v2_fully_annotated_annotation.zip
drwxr-xr-x 3 root root 4.0K Jan 31 03:19  mtsd_v2_fully_annotated_images.val.zip
-rw-r--r-- 1 root root  26M Jan 31 03:20  output_file_coco_new.json
-rw-r--r-- 1 root root  61M Jan 31 03:20  train.json
-rw-r--r-- 1 root root 8.7M Jan 31 03:20  val.json
-rw-r--r-- 1 root root 3.8M Jan 31 03:20  val_output_file_coco_new.json


In [12]:
!ls -lh /content

total 16K
drwx------ 5 root root 4.0K Jan 31 01:07 drive
-rw-r--r-- 1 root root   85 Jan 31 02:06 kaggle.json
drwxr-xr-x 2 root root 4.0K Jan 31 00:32 mtsd_work
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data


In [13]:
SRC_PATH = "/root/.cache/kagglehub/datasets/zeuss2k3/mapillary-traffic-sign-dataset/versions/8"

WORKDIR = "/content/mtsd_work"

!mkdir -p "$WORKDIR"
!cp -r "$SRC_PATH"/* "$WORKDIR"

print("✅ Dataset copied to WORKDIR")
!ls "$WORKDIR"


✅ Dataset copied to WORKDIR
'annotations for mtsd in coco format'	  output_file_coco_new.json
 mtsd_fully_annotated_train_images	  train.json
 mtsd_partially_annotated_annotation	  val.json
 mtsd_v2_fully_annotated_annotation.zip   val_output_file_coco_new.json
 mtsd_v2_fully_annotated_images.val.zip


In [10]:
from pathlib import Path

root = Path(WORKDIR)

def find_one(pattern):
    hits = list(root.rglob(pattern))
    return hits[0] if hits else None

TRAIN_JSON = find_one("train.json")
VAL_JSON   = find_one("val.json")

TRAIN_IMG_DIR = find_one("mtds_fully_annotated_train_images")
VAL_IMG_DIR   = find_one("mtds_v2_fully_annotated_images.val")

print("TRAIN_JSON:", TRAIN_JSON)
print("VAL_JSON:  ", VAL_JSON)
print("TRAIN_IMG_DIR:", TRAIN_IMG_DIR)
print("VAL_IMG_DIR:  ", VAL_IMG_DIR)

assert TRAIN_JSON and VAL_JSON, "❌ Nuk po gjej train.json / val.json. Kontrollo ku janë brenda WORKDIR."
assert TRAIN_IMG_DIR and VAL_IMG_DIR, "❌ Nuk po gjej folderat e imazheve. Kontrollo emrat."

TRAIN_JSON: None
VAL_JSON:   None
TRAIN_IMG_DIR: None
VAL_IMG_DIR:   None


AssertionError: ❌ Nuk po gjej train.json / val.json. Kontrollo ku janë brenda WORKDIR.

In [ ]:
import os

YOLO_ROOT = Path("/content/yolo_data")
train_img_out = YOLO_ROOT / "images/train"
val_img_out   = YOLO_ROOT / "images/val"
train_lbl_out = YOLO_ROOT / "labels/train"
val_lbl_out   = YOLO_ROOT / "labels/val"

for p in [train_img_out, val_img_out, train_lbl_out, val_lbl_out]:
    p.mkdir(parents=True, exist_ok=True)

# Symlink images (nëse ekziston, e heqim dhe e bëjmë prapë)
def make_link(src, dst):
    if dst.exists() or dst.is_symlink():
        dst.unlink()
    os.symlink(src, dst)

make_link(str(TRAIN_IMG_DIR), str(train_img_out))
make_link(str(VAL_IMG_DIR), str(val_img_out))

print("✅ YOLO folder ready at:", YOLO_ROOT)
print("Train images link:", train_img_out, "->", os.readlink(train_img_out))
print("Val images link:  ", val_img_out, "->", os.readlink(val_img_out))


In [ ]:
import json
from tqdm import tqdm

def coco_to_yolo_labels(coco_json_path, labels_out_dir):
    labels_out_dir = Path(labels_out_dir)
    labels_out_dir.mkdir(parents=True, exist_ok=True)

    with open(coco_json_path, "r", encoding="utf-8") as f:
        coco = json.load(f)

    images = {img["id"]: img for img in coco["images"]}
    categories = sorted(coco["categories"], key=lambda c: c["id"])

    # COCO category_id nuk është gjithmonë 1..N, prandaj bëjmë map stabil
    cat_id_to_yolo = {cat["id"]: i for i, cat in enumerate(categories)}
    names = [cat["name"] for cat in categories]

    print(f"📌 {Path(coco_json_path).name}: images={len(images)}, annotations={len(coco['annotations'])}, classes={len(names)}")

    # Pastro labels folder (opsionale)
    for fpath in labels_out_dir.glob("*.txt"):
        fpath.unlink()

    for ann in tqdm(coco["annotations"]):
        img = images.get(ann["image_id"])
        if img is None:
            continue

        img_w, img_h = img["width"], img["height"]
        x, y, w, h = ann["bbox"]

        # normalize
        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w_norm = w / img_w
        h_norm = h / img_h

        cls = cat_id_to_yolo[ann["category_id"]]

        label_path = labels_out_dir / f"{Path(img['file_name']).stem}.txt"
        with open(label_path, "a", encoding="utf-8") as out:
            out.write(f"{cls} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

    return names

names_train = coco_to_yolo_labels(TRAIN_JSON, train_lbl_out)
names_val   = coco_to_yolo_labels(VAL_JSON, val_lbl_out)

# Kontroll: a janë njësoj listat e klasave?
print("Same class list train/val:", names_train == names_val)
print("Total classes:", len(names_train))


In [ ]:
import yaml

data_yaml = {
    "path": str(YOLO_ROOT),
    "train": "images/train",
    "val": "images/val",
    "names": {i: n for i, n in enumerate(names_train)}
}

yaml_path = YOLO_ROOT / "data.yaml"
with open(yaml_path, "w", encoding="utf-8") as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False, allow_unicode=True)

print("✅ Wrote:", yaml_path)
print("Example names:", list(data_yaml["names"].items())[:5])


In [ ]:
import random, cv2
import matplotlib.pyplot as plt

def show_random_example(split="train"):
    img_dir = YOLO_ROOT / f"images/{split}"
    lbl_dir = YOLO_ROOT / f"labels/{split}"

    # gjej një imazh rastësor
    img_paths = list(img_dir.rglob("*.jpg")) + list(img_dir.rglob("*.png")) + list(img_dir.rglob("*.jpeg"))
    img_path = random.choice(img_paths)

    lbl_path = lbl_dir / (img_path.stem + ".txt")

    img = cv2.imread(str(img_path))
    h, w = img.shape[:2]

    if lbl_path.exists():
        with open(lbl_path, "r") as f:
            lines = f.read().strip().splitlines()

        for line in lines[:20]:  # max 20 boxes vetëm për vizualizim
            cls, xc, yc, bw, bh = line.split()
            cls = int(cls)
            xc, yc, bw, bh = map(float, (xc, yc, bw, bh))

            x1 = int((xc - bw/2) * w)
            y1 = int((yc - bh/2) * h)
            x2 = int((xc + bw/2) * w)
            y2 = int((yc + bh/2) * h)

            cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(img, names_train[cls], (x1, max(20, y1-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10,7))
    plt.title(f"{split.upper()} sample: {img_path.name}")
    plt.imshow(img_rgb)
    plt.axis("off")

show_random_example("train")
show_random_example("val")


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # ose "yolov8s.pt"

results = model.train(
    data=str(yaml_path),
    epochs=30,        # nis me 30; pastaj rrit në 50-100
    imgsz=640,
    batch=16,
    workers=2,
    device=0
)

In [ ]:
from ultralytics import YOLO
import cv2, random
import matplotlib.pyplot as plt

best_model = YOLO("runs/detect/train/weights/best.pt")

# merr një imazh random nga val
val_imgs = list((YOLO_ROOT / "images/val").rglob("*.jpg")) + list((YOLO_ROOT / "images/val").rglob("*.png"))
img_path = random.choice(val_imgs)

pred = best_model.predict(source=str(img_path), conf=0.4, verbose=False)[0]
img = pred.plot()  # BGR image me boxes

plt.figure(figsize=(10,7))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Prediction sample")
plt.show()

print("Used image:", img_path.name)

In [ ]:
from google.colab import files
files.download("runs/detect/train/weights/best.pt")